In [13]:
import pandas as pd
import torch

from predict import Bearing_Predictor
from predict import calculate_data

In [14]:
num_heads = 2
num_encoder_layers = 6
dim_feedforward = 256
dropout = 0.1
#模型创建
model_lodar = Bearing_Predictor(
    feature_dim=18,
    num_heads=num_heads,
    num_encoder_layers=num_encoder_layers,
    dim_feedforward=dim_feedforward,
    dropout=dropout,
)
# model_lodar.load_state_dict(torch.load("bearing_predictor.pth",map_location='cpu'))

model_lodar = torch.load('bearing_predictor.pth')
model_lodar.eval()

Bearing_Predictor(
  (prenet): Linear(in_features=18, out_features=18, bias=True)
  (encoderlayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=18, out_features=18, bias=True)
    )
    (linear1): Linear(in_features=18, out_features=256, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=256, out_features=18, bias=True)
    (norm1): LayerNorm((18,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((18,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=18, out_features=18, bias=True)
        )
        (linear1): Linear(in_features=18, out_features=256, bias=True)
        (d

In [15]:
f = open(r"phm-ieee-2012-data-challenge-dataset-master\Learning_set\Bearing1_1\acc_00001.csv", encoding="utf-8")
data_reader = pd.read_csv(f, header=None)
data_reader.columns = [
    "hour",
    "minute",
    "second",
    "micro",
    "Horizontal_acceleration",
    "Vertical_acceleration",
]
data = data_reader.copy()
data

,hour,minute,second,micro,Horizontal_acceleration,Vertical_acceleration
0,9,39,39,65664.0,0.552,-0.146
1,9,39,39,65703.0,0.501,-0.480
2,9,39,39,65742.0,0.138,0.435
3,9,39,39,65781.0,-0.423,0.240
4,9,39,39,65820.0,-0.802,0.020
...,...,...,...,...,...,...
2555,9,39,39,165470.0,0.044,-0.094
2556,9,39,39,165510.0,-0.456,0.486
2557,9,39,39,165550.0,-0.885,-0.154
2558,9,39,39,165580.0,-0.230,-0.762


In [16]:
reason_data = pd.read_csv('datas/轴承A.csv', encoding="gbk")
reason_data = reason_data.copy().drop(columns=['X轴加速度(g)', 'Y轴加速度(g)', '采样频率(Hz)']).rename(
    columns={'Z轴加速度(g)': 'Vertical_acceleration'})
data_seq = []
for i in range(0, 10):
    data_seq.append(calculate_data(reason_data[i * 2600:(i + 1) * 2600], freq=1 / 26471))  #分10次取2600条样本
bearing_data_sequence = torch.stack(data_seq, dim=0).float()
bearing_data_sequence

tensor([[ 1.9213e-01,  5.6040e-02,  3.1400e-03,  4.4897e-02,  3.5158e-01,
          3.8214e-02, -1.0054e-01,  1.2172e-01,  1.2482e+00,  3.4284e+00,
          4.2794e+00,  5.0277e+00,  8.1654e+00,  1.8896e-05,  4.6513e-07,
          2.8575e+00,  5.1470e-05,  1.8883e-05],
        [ 2.0373e-01,  5.1085e-02,  2.6094e-03,  3.9085e-02,  3.5274e-01,
          3.2246e-02,  8.0760e-01,  3.5112e-01,  1.3070e+00,  3.9881e+00,
          5.2125e+00,  6.3181e+00,  6.7852e+00,  1.8896e-05,  4.6513e-07,
          2.6048e+00,  4.6696e-05,  1.8886e-05],
        [ 2.0714e-01,  5.2152e-02,  2.7194e-03,  4.1649e-02,  3.6075e-01,
          3.5429e-02,  1.0262e-01,  1.7012e-01,  1.2522e+00,  3.9717e+00,
          4.9733e+00,  5.8465e+00,  7.0716e+00,  1.8896e-05,  4.6513e-07,
          2.6593e+00,  4.7681e-05,  1.8885e-05],
        [ 1.8287e-01,  5.4439e-02,  2.9635e-03,  4.2797e-02,  3.4518e-01,
          3.5734e-02,  8.2685e-02,  1.3272e-01,  1.2720e+00,  3.3592e+00,
          4.2730e+00,  5.1175e+00,  7.7

In [17]:
bearing_data_sequence = bearing_data_sequence.unsqueeze(0)
pridict = model_lodar(bearing_data_sequence)  #传入数据的结构是什么

In [18]:
pridict

tensor([[[0.7811]],

        [[0.7799]],

        [[0.7771]],

        [[0.7814]],

        [[0.7826]],

        [[0.7791]],

        [[0.7779]],

        [[0.7836]],

        [[0.7824]],

        [[0.7794]]], grad_fn=<ViewBackward0>)